# SQL Query

*Easily search through the lighting database with this script*

## Import of libraries needed for sql search, dataframe use, and html plot

In [1]:
import pyodbc
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Category20
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
import itertools

import numpy as np

from datetime import date, timedelta, datetime

## Database Connection
*Name of server, database, utilizes ad3 user and password from computer*

In [2]:
server = 'cfo-sql1' #name of the sql server 
database = 'Lighting' #name of the database in the sql server
username = '' #will use ad3 user and password for computer 
password =  ''

#Command for connecting to sql server using pyodbc connect
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password) 
cursor = cnxn.cursor()

## Inputs: Building Name, Measure Type, Start Date, End Date, Frequency
*Building Example Inputs: 'Kemper', 'Ghausi', 'Meyer', 'Gourley', 'Hart'...*

*Measure Type Examples: 'KW', 'Daylight', 'Occupancy', 'Switchlock', 'Afterhours'*

In [3]:
buildings_input = [
#'ACADEMICSURGE'
#'DUTTONHALL'
#'GHAUSI'
'GOURLEYCLINICALCNTR'
#'HARTHALL'
#'KEMPER'
#'LIFESCIENCE'
#'MATHSCIENCE'
#'MEYERHALL'
#'PLANTENVIROSCIENCE'
#'PLANTREPROBIOFAC'
#'SOCIALSCIENCES'
#'SCIENCESLAB'
#'VETMED3A'
]

room_desired = input("Room Number?"" 'Y' or 'N': ")
if room_desired != 'N':
    room_number_input = input("Enter Room Number: ")

meastype_input = [
#'AFTERHOURS'
#'AREA'
#'DAYLIGHT'
'KW' 
#'LKW'
#'OCCUPANCY'
#'PKW'
#'SWITCHLOCK'
#'TKW'
#'WFT'
]

Room Number? 'Y' or 'N': N


*Required Date Input Format (mm/dd/yyyy)*

time_range = "specific date" ~ able to enter specific date range

In [4]:
time_range = "week-to-date"
#"specificdate"
#"today"
#"last24hrs"
#"yesterday"
#"week-to-date"
#"lastweek"
#"last7days"
#"month-to-date"
#"lastmonth"
#"year-to-date"
#"lastyear"

today = date.today()
index = today.weekday() #monday starts at 0

if time_range == "specificdate":
    start_date = input("Enter Start Date: ")
    end_date = input("Enter End Date: ")
elif time_range == "today": 
    start_date = today
    end_date = today + timedelta(days = 1)
elif time_range == "last24hrs":
    end_date = datetime.now()
    start_date = datetime.now() - timedelta(hours = 24)
    end_date = end_date.replace(second = 0, microsecond = 0)
    start_date = start_date.replace(second = 0, microsecond = 0)
elif time_range == "yesterday":
    end_date = today
    start_date = today - timedelta(days = 1) 
elif time_range == "week-to-date":
    start_date = today - timedelta(1+index)
    end_date = today + timedelta(days = 1)
elif time_range == "lastweek":
    start_date = today - timedelta(8 + index)
    end_date = today - timedelta(1 + index)
elif time_range == "last7days":
    start_date = today - timedelta(days = 7)
    end_date = today + timedelta(days = 1)
elif time_range == "month-to-date": #needa fix
    start_date = today.replace(day = 1) 
    end_date = today + timedelta(days = 1)
elif time_range == "lastmonth":
    end_date = today.replace(day = 1) 
    start_date = end_date - timedelta(days = 31)
elif time_range == "year-to-date":
    start_date = "1/1/2019"
    end_date = today + timedelta(days = 1)
elif time_range == "lastyear":
    end_date = date(date.today().year, 1, 1) - timedelta(days = 1)
    start_date = date(end_date.year, 1, 1)
    end_date = end_date + timedelta(days = 1)
    
start_date = str(start_date)
end_date = str(end_date)

*Frequency Examples ~ 'D': Daily, 'H': Hourly, '15min': Every 15 minutes (whatever #min desired)*

In [5]:
frequency = input("Enter frequency: ")

Enter frequency: 15min


# Collecting tables that meet sql search criteria

*Searches through database based on building name and measure type*

In [6]:
building_name = []
meastype = []
room_number = []

if room_desired != 'N':
    for name in buildings_input:
        for measure in meastype_input:
            for room in room_number_input:
                building_name = name
                meastype = measure
                room_number = room
                filtered_array = []
                read_sql = "SELECT * from Sys.Tables WHERE name like '%_" + meastype + "%' and name like '%_" + room_number_input + "%' and name like '%" + building_name + "%'"
else:
    for name in buildings_input:
        for measure in meastype_input:
            building_name = name 
            meastype = measure
            filtered_array = []
            read_sql = "SELECT * from Sys.Tables WHERE name like '%_" + meastype + "%' and name like '%" + building_name + "%'" 

*Creates a dataframe with table names that meet criteria*

In [7]:
    with cursor.execute(read_sql):
        row = cursor.fetchone() #retrieves next row of query result set and returns single sequence
        while row:
            filtered_array.append({'table': row[0]})
            row = cursor.fetchone()
        
    #converting table into dataframe with table names of sql search that meets criteria
    filtered_dataframe = pd.DataFrame(filtered_array) 
    display(filtered_dataframe)

,table
0,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_LKW
1,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_PKW
2,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_TKW
3,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_LKW
4,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_PKW
5,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_TKW
6,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_LKW
7,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_PKW
8,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_TKW
9,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1129_LKW


*Searching specific date and frequency through filtered dataframe*

In [8]:
    timerange = pd.date_range(start = start_date, end = end_date, freq = frequency)
    final_dataframe = pd.DataFrame()

    neveroff = pd.DataFrame(columns=['Room', 'Average', 'Standard Div'])
    
    for column in filtered_dataframe['table']:
        building = []
        read_sql = "SELECT [TIMESTAMP],[VALUE] from [dbo].["+column+"] WHERE [TIMESTAMP] BETWEEN "  + "'" + start_date +"'" + " AND " + "'"+end_date+"'"
        with cursor.execute(read_sql):
            row = cursor.fetchone()
            while row:
                building.append({'date':row[0], column:row[1]})
                row = cursor.fetchone()
        building = pd.DataFrame(building)
        try: #when table column is empty
            building = building.set_index('date') #sets date column as index
            #filter to minute resolution
            building.index = building.index.map(lambda x: x.replace(second=0)) 
            building.index = building.index.map(lambda x: x.replace(microsecond=0))
            #filtering in case of duplicates
            building = building[~building.index.duplicated(keep='first')]
            
            #occupancy sensor query
            if meastype == 'OCCUPANCY':
                building = building.reindex(timerange)
            else:
                building = building.resample(frequency).backfill()
                
                
            final_dataframe = pd.concat([building, final_dataframe], axis=1)
            #print(column)
            
            #neveroff calculation
            averagevalue = building.mean(). iloc[0]
            maxvalue = building.max(). iloc[0]
            stdiv = np.std([averagevalue, maxvalue])
            if (averagevalue != 0) and (stdiv < .01):
                neveroff = neveroff.append({'Room':column, 'Average':averagevalue,'Standard Div':stdiv},ignore_index=True)
        except:
            pass
display(neveroff)

,Room,Average,Standard Div
0,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1145_LKW,0.087,0.0


*Turns Boolean into '1' for on and '0' for off*

In [9]:
    final_dataframe *=1

*Outputs to CSV file*

In [10]:
    final_dataframe.columns = sorted(final_dataframe.columns, key=lambda item: (int(item.partition(' ')[0]) if item[0].isdigit() else float('inf'), item))
    start_date = start_date.replace('/', '_')
    buildings_input = str(buildings_input).replace('[','').replace(']','')
    meastype_input = str(meastype_input).replace('[','').replace(']','')

    pd.set_option('display.max_rows', 5000)
    pd.set_option('display.max_columns', 5000)
    
    if time_range == "last24hrs":
        start_date = today - timedelta(days = 1) 
    else:
        if room_desired != 'N':
            final_dataframe.to_csv(buildings_input+'_'+room_number_input+'_'+meastype_input+'_DATA_'+start_date+'.csv') #exports to csv and appears in file explorer
        else:
            final_dataframe.to_csv(buildings_input+'_'+meastype_input+'_DATA_'+start_date+'.csv')
            
    display(final_dataframe)

,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1129_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130A_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130B_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1131_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1133_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1135_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1137_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1140_2_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1140_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1143A_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1143_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1145_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1147_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1150_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1155_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1156A_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1156_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1157_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1159_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160A_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160B_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160C_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160D_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1180_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1195_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1200_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1202_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1204_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1214_LKW,UCDAVIS_GOURLEYCLINICALCNTR_LKW
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-11 00:00:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,2.981,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 00:15:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,2.981,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 00:30:00,3.090,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,3.003,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 00:45:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,2.981,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 01:00:00,3.090,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,3.003,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 01:15:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 01:30:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 01:45:00,3.068,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000
2019-08-11 02:00:00,3.112,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.000,3.025,0.00,0.000,0.000,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000


## Plots data on HTML file with Bokeh Plotting

*Asks user if they want to plot the data*

In [12]:
    plot_desired = input("Plot? 'Y' or 'N': ")
    
    if plot_desired != 'N':
         def color_gen():
            for c in itertools.cycle(Category20[10]):
                yield c
         
         #fix date for hover
         #add time range for title or something
         hover = HoverTool(tooltips=[('date', '@index'), (meastype, '$y')], formatters = {'date':'datetime'}, mode = 'vline')
         
        
         if room_desired != 'N':
            graph = graph = figure(plot_width = 1000, plot_height=800, x_axis_type = 'datetime', title = buildings_input+'_'+room_number_input+'_'+meastype_input+'_DATA_'+start_date)
         else:
            graph = graph = figure(plot_width = 1000, plot_height=800, x_axis_type = 'datetime', title = buildings_input+'_'+meastype_input+'_DATA_'+start_date)
            
         graph.xaxis.axis_label = 'Date'
         graph.yaxis.axis_label = meastype
    
         colors = color_gen()
         tags = final_dataframe.columns
         for tag in tags:
             color = next(colors)
             graph.circle(final_dataframe.index, final_dataframe[tag], size = 10, color = color, legend = tag)
             graph.line(final_dataframe.index, final_dataframe[tag], color = color, legend = tag)
            
         graph.legend.click_policy = 'hide'
         graph.legend.location = 'top_right'
         #fix legend to hide it if you want 
        
         graph.add_tools(hover)
         
         if room_desired != 'N':
            output_file(buildings_input+'_'+meastype_input+'_'+room_number_input+'_DATA_'+start_date+'.html')
         else:
             output_file(buildings_input+'_'+meastype_input+'_DATA_'+start_date+'.html')
        
         output_notebook()
         show(graph)

Plot? 'Y' or 'N': Y


Loading BokehJS ...